In [39]:
import pandas as pd
import yfinance as yf
from datetime import datetime
from fredapi import Fred
from plotly.subplots import make_subplots
import plotly.graph_objs as go

# FRED API Key
fred_api_key = 'ecc0c4e92352e1eff5869528238a3bff'

def yfinance_data(ticker, start, end):
    try:
        return yf.download(ticker, start=start, end=end)
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return pd.DataFrame()  # Return an empty DataFrame in case of error

def fred_data(series_id):
    fred = Fred(api_key=fred_api_key)
    try:
        data = fred.get_series(series_id)
        return data
    except Exception as e:
        print(f"Error fetching data from FRED: {e}")
        return pd.Series()  # Return an empty Series in case of error

In [40]:
# Define date range
start_date = '2008-01-01'
end_date = datetime.today().strftime('%Y-%m-%d')

# Fetch ETH-USD data
eth_df = yfinance_data('ETH-USD', start=start_date, end=end_date)
eth_df.head(), eth_df.info()

# Fetch BTC-USD data
btc_df = yfinance_data('BTC-USD', start=start_date, end=end_date)
btc_df.head(), btc_df.info()

# Fetch S&P 500 data
sp500_df = yfinance_data('^GSPC', start=start_date, end=end_date)
sp500_df.head(), sp500_df.info()

# Fetch M2 Money Supply data from FRED
m2_series = fred_data('M2SL')
m2_series.head()  # Inspect the first few rows of the M2 data


DEBUG:yfinance:Entering download()
DEBUG:yfinance:Disabling multithreading because DEBUG logging enabled
DEBUG:yfinance: Entering history()
DEBUG:peewee:('SELECT "t1"."key", "t1"."value" FROM "_kv" AS "t1" WHERE ("t1"."key" = ?) LIMIT ? OFFSET ?', ['ETH-USD', 1, 0])
DEBUG:yfinance:  Entering history()
DEBUG:yfinance:ETH-USD: Yahoo GET parameters: {'period1': '2008-01-01 00:00:00+00:00', 'period2': '2024-08-24 00:00:00+00:00', 'interval': '1d', 'includePrePost': False, 'events': 'div,splits,capitalGains'}
DEBUG:yfinance:ETH-USD: yfinance received OHLC data: 2017-11-09 00:00:00 -> 2024-08-23 00:00:00
DEBUG:yfinance:ETH-USD: OHLC after cleaning: 2017-11-09 00:00:00+00:00 -> 2024-08-23 00:00:00+00:00
DEBUG:yfinance:ETH-USD: OHLC after combining events: 2017-11-09 00:00:00+00:00 -> 2024-08-23 00:00:00+00:00
DEBUG:yfinance:ETH-USD: yfinance returning OHLC: 2017-11-09 00:00:00+00:00 -> 2024-08-23 00:00:00+00:00
DEBUG:yfinance:  Exiting history()
DEBUG:yfinance: Exiting history()
DEBUG:yfinanc

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2480 entries, 2017-11-09 to 2024-08-23
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       2480 non-null   float64
 1   High       2480 non-null   float64
 2   Low        2480 non-null   float64
 3   Close      2480 non-null   float64
 4   Adj Close  2480 non-null   float64
 5   Volume     2480 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 135.6 KB
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3629 entries, 2014-09-17 to 2024-08-23
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       3629 non-null   float64
 1   High       3629 non-null   float64
 2   Low        3629 non-null   float64
 3   Close      3629 non-null   float64
 4   Adj Close  3629 non-null   float64
 5   Volume     3629 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 198.5 KB
<class '

1959-01-01    286.6
1959-02-01    287.7
1959-03-01    289.2
1959-04-01    290.1
1959-05-01    292.2
dtype: float64

In [41]:
# Convert M2 Series to DataFrame and align with S&P 500
m2_df = pd.DataFrame(m2_series, columns=['M2'])
m2_df.index = pd.to_datetime(m2_df.index)  # Ensure the index is in datetime format
m2_df = m2_df.loc[start_date:end_date]  # Filter M2 data to match the date range
m2_df['M2'] = m2_df['M2'] / 1e9  # Convert to billions

# Inspect the M2 data after transformation
m2_df.head(), m2_df.info()

# Resample S&P 500 data to match the monthly frequency of M2 data
sp500_df = sp500_df.resample('ME').last()  # Resample to monthly frequency using the last value of each month
sp500_df['Close'] = pd.to_numeric(sp500_df['Close'], errors='coerce')

# Inspect the S&P 500 data after resampling
sp500_df.head(), sp500_df.info()


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 198 entries, 2008-01-01 to 2024-06-01
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   M2      198 non-null    float64
dtypes: float64(1)
memory usage: 3.1 KB
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 200 entries, 2008-01-31 to 2024-08-31
Freq: ME
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       200 non-null    float64
 1   High       200 non-null    float64
 2   Low        200 non-null    float64
 3   Close      200 non-null    float64
 4   Adj Close  200 non-null    float64
 5   Volume     200 non-null    int64  
dtypes: float64(5), int64(1)
memory usage: 10.9 KB


(                   Open         High          Low        Close    Adj Close  \
 Date                                                                          
 2008-01-31  1351.979980  1385.619995  1334.079956  1378.550049  1378.550049   
 2008-02-29  1364.069946  1364.069946  1325.420044  1330.630005  1330.630005   
 2008-03-31  1315.920044  1328.520020  1312.810059  1322.699951  1322.699951   
 2008-04-30  1391.219971  1404.569946  1384.250000  1385.589966  1385.589966   
 2008-05-31  1398.359985  1404.459961  1398.079956  1400.380005  1400.380005   
 
                 Volume  
 Date                    
 2008-01-31  4970290000  
 2008-02-29  4426730000  
 2008-03-31  4188990000  
 2008-04-30  4508890000  
 2008-05-31  3845630000  ,
 None)

In [42]:
# Merge M2 and S&P 500 data
m2_sp500_df = pd.concat([m2_df, sp500_df['Close']], axis=1)
m2_sp500_df.columns = ['M2 (Billions)', 'S&P 500']  # Flatten columns and label appropriately

# Inspect the merged DataFrame
m2_sp500_df.head(), m2_sp500_df.info()


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 398 entries, 2008-01-01 to 2024-08-31
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   M2 (Billions)  198 non-null    float64
 1   S&P 500        200 non-null    float64
dtypes: float64(2)
memory usage: 9.3 KB


(            M2 (Billions)      S&P 500
 2008-01-01       0.000008          NaN
 2008-01-31            NaN  1378.550049
 2008-02-01       0.000008          NaN
 2008-02-29            NaN  1330.630005
 2008-03-01       0.000008          NaN,
 None)